# Generate SSH Keys on [Commjhub](https://commjhub.asc.upenn.edu/) for GitHub 🚀🔑⚡

Etienne P Jacquot - [epj@asc.upenn.edu](mailto:epj@asc.upenn.edu)

_______

## Getting Started

Following instructions here: https://docs.github.com/en/github/authenticating-to-github/generating-a-new-ssh-key-and-adding-it-to-the-ssh-agent

We do *NOT* want to have undergraduate students entering passwords in plain text!!! SSH is preferred!


### *EXAMPLE: We run this in *one* bash line, instead of multiple python cells:*

``` bash

ssh-keygen -t ed25519 -C "GITHUB_USER@commjhub.asc.upenn.edu" -f $HOME/.ssh/id_rsa -N "" <<< y
eval "$(ssh-agent -s)"
ssh-add $HOME/.ssh/id_rsa -vvv

```

______

### To create your SSH key and add to your SSH-Agent:


- Default directory is for `$HOME/.ssh/id_rsa`


- This defaults to *NO* password with the `-N ""` flag


- We include the `<<< y` to overwrite an existing key. Be careful to not rerun this cell *after* uploading your SSH key to github as it'll overwrite!

In [1]:
!ssh-keygen -t ed25519 -C "atnjqt@commjhub.asc.upenn.edu" -f $HOME/.ssh/id_rsa -N "" <<< y && \
eval "$(ssh-agent -s)" && \
ssh-add $HOME/.ssh/id_rsa

Generating public/private ed25519 key pair.
/Commjhub/jupyterhub/comm318_fall2019/{atnjqt}/.ssh/id_rsa already exists.
Overwrite (y/n)? Your identification has been saved in /Commjhub/jupyterhub/comm318_fall2019/{atnjqt}/.ssh/id_rsa.
Your public key has been saved in /Commjhub/jupyterhub/comm318_fall2019/{atnjqt}/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:kzp2Um4n7SXhQrdQU5zqfnvJ0RmlHoOchFlSEPDGFnY atnjqt@commjhub.asc.upenn.edu
The key's randomart image is:
+--[ED25519 256]--+
|        ..*OE.   |
|         +o=+   .|
|          B+ o ..|
|         =..+ +. |
|        S.o  . +o|
|       = *.o  o..|
|      = *.* .. o |
|     . = =.o. +  |
|          ...o   |
+----[SHA256]-----+
Agent pid 19946
Identity added: /Commjhub/jupyterhub/comm318_fall2019/{atnjqt}/.ssh/id_rsa (atnjqt@commjhub.asc.upenn.edu)


_______
### Proceed by copying your public key

- Remember to *NEVER* share your private key `.ssh/id_rsa`!

- You need to copy the full result to your clipboard:

In [2]:
!cat $HOME/.ssh/id_rsa.pub

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAILX++IP5T4fAs6YDqT6VIUdjKWnsP095ADr1ljuFGj8p atnjqt@commjhub.asc.upenn.edu


### Upload your public key to Github

Following the instructions here: https://docs.github.com/en/github/authenticating-to-github/adding-a-new-ssh-key-to-your-github-account 

- This is a manual step that you must do on your https://github.com/ login... For example:


<img width='700' src='https://imghost.asc.upenn.edu/documentation/commjhub_ssh-pub-key.png' alt='Screenshot of Github New SSH Public Key configuration' style="vertical-align:middle;margin:0px 50px"/> 


___________

## Testing your SSH Access to Github.com

Once your SSH public key is uploaded to github, please follow instructions here to securely test your connection: https://docs.github.com/en/github/authenticating-to-github/testing-your-ssh-connection


### Prepare your Github known hosts file:

--> *Manually navigate to get the **RSA public key fingerprint**: https://docs.github.com/en/github/authenticating-to-github/githubs-ssh-key-fingerprints*


In [3]:
# ENTER THIS VALUE FROM OFFICIAL GITHUB WEBSITE
rsa_pubkey_fingerprint = 'SHA256:nThbg6kXUpJWGl7E1IGOCspRomTxdCARLviKw6E5SY8'

### Get SSH host information & RSA fingerprint over the internet

First scan & write the host information for *github.com* to `./github_ssh_test`. Next we get the rsa fingerprint for that respective scanned host info and write to `./fingerprint_rsa` to then check against our known trusted value.

- *This example is based on a helpful serverfault question here: https://serverfault.com/questions/856194/securely-add-a-host-e-g-github-to-the-ssh-known-hosts-file*


In [4]:
!ssh-keyscan -t rsa github.com | tee ./github_ssh_test | ssh-keygen -lf - >> fingerprint_rsa

# github.com:22 SSH-2.0-babeld-0913e7e1


### Confirm known RSA fingerprint matches SSH-Keyscan'd fingerprint and write to `Known_Hosts`

We use a python assertion to ensure authenticity of our known hosts.

- Alternatively, we can add the known host to `/etc/ssh/ssh_known_hosts` as system admin!

In [5]:
assert(rsa_pubkey_fingerprint == open("./fingerprint_rsa", "r").read().split()[1])

!cat ./github_ssh_test >> $HOME/.ssh/known_hosts

#### *If the above assertion fails...*

Then the value was *not* added to your known hosts file... Try to resolve with:

- confirm you set the correct `rsa_pubkey_fingerprint` from [here](https://docs.github.com/en/github/authenticating-to-github/githubs-ssh-key-fingerprints)
- confirm there is no man in the middle attack for github.com on your local network (you have bigger problems if that is the case!)...
________________

## Confirm your SSH connectivity to Github

- We are able to avoid needing the `-o "StrictHostKeyChecking no"` flag as you cannot respond *yes* to this in stdout. It is like that by design for security!

In [6]:
# You can always confirm your access know that the known host is added:
#!ssh -o "StrictHostKeyChecking no" -T git@github.com
!ssh -T git@github.com

Hi atnjqt! You've successfully authenticated, but GitHub does not provide shell access.


___________

## Set your global git configuration on Commjhub:

- You must include your `firstname`, `lastname`, and `email`. These are really just unique identifiers so please set to something meaningful.

- Include the `--replace-all` if you want to overwrite all your git configurations. 


### Your personal information here:

In [7]:
firstname = "Etienne"
lastname = "Jacquot"
email = 'jacquot.etienne@gmail.com'

### Set your Git Global configurations:

In [8]:
my_name = '"{} {}"'.format(firstname,lastname) # <--- formatting name

!git config --global --replace-all user.name $my_name

!git config --global --replace-all user.email $email

!git config --global push.default simple # <--- for older versions of git

### Confirm your git configurations:

In [9]:
!git config --list

user.email=jacquot.etienne@gmail.com
user.name=Etienne Jacquot
push.default=simple


_______________

## SSH Clone a private git repo to make changes:

I have an old private git repo here: https://github.com/atnjqt/ASC_Cryptominer

- For this to work, you need *your own private git repository!*

In [11]:
!git clone git@github.com:atnjqt/ASC_Cryptominer.git

Cloning into 'ASC_Cryptominer'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 50 (delta 19), reused 13 (delta 3), pack-reused 0
Receiving objects: 100% (50/50), 3.61 MiB | 0 bytes/s, done.
Resolving deltas: 100% (19/19), done.


__________

## Manually go into your git repo, make a change, then run the below cells to add your commit & push

In my case my git repo is of course: [ASC_Cryptominer/](./ASC_Cryptominer/)


- **Please note:** we include `!cd your_private_repo_name/` at the beginning of each cmd. This is just to *change directory* so our git commands are referencing the correct directory

### Check the status to confirm your changes:

Should have something like

```
# On branch master
# Changes not staged for commit:
#   (use "git add <file>..." to update what will be committed)
#   (use "git checkout -- <file>..." to discard changes in working directory)
#
#	modified:   THE_FILE_THAT_YOU_CHANGED.txt
#
no changes added to commit (use "git add" and/or "git commit -a")

```

In [12]:
!cd ASC_Cryptominer/ && \
git status

# On branch master
# Changes not staged for commit:
#   (use "git add <file>..." to update what will be committed)
#   (use "git checkout -- <file>..." to discard changes in working directory)
#
#	modified:   COMMJHUB_SSH_TEST/TESTING.txt
#
no changes added to commit (use "git add" and/or "git commit -a")


### Add and Commit your changes for the private git repo

- Always include a meaningful message!

In [13]:
!cd ASC_Cryptominer && \
git add --all && \
git commit -m 'ssh key testing changes for private repo'

[master 13abd37] ssh key testing changes for private repo
 1 file changed, 4 insertions(+), 1 deletion(-)


### Finally, push your changes to a private git repository:


In [14]:
!cd ASC_Cryptominer/ && \
git push -v

Pushing to git@github.com:atnjqt/ASC_Cryptominer.git
Counting objects: 7, done.
Delta compression using up to 16 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 426 bytes | 0 bytes/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To git@github.com:atnjqt/ASC_Cryptominer.git
   c55fbfe..13abd37  master -> master
updating local tracking ref 'refs/remotes/origin/master'


### If that git push fails...


- If you are using branches or get an error, try `git push origin master`


- If you get a warning about `github push.default is unset behavior`... just run (included above so you shouldn't get this, probably git needs to be upgraded on Commjhub VM):

``` bash
git config --global push.default simple
```

______

- If you are still having issues, send me an email at [epj@asc.upenn.edu](mailto:epj@asc.upenn.edu) 🚀

<img width='550' src='https://imghost.asc.upenn.edu/gifs/atn_zoomba.gif'/> 